## Prédiction de la valeur d'un vol avec le Boosting

Kaggle : https://www.kaggle.com/datasets/shubhambathwal/flight-price-prediction

Nous travaillerons avec les datasets brut, pas ceux travaillé par les personnes qui l'ont publié sur Kaggle. Il y aura deux datasets que l'on va utiliser ensemble.

In [1]:
import numpy as np
import pandas as pd


economy = pd.read_csv("economy.csv")
economy["Type"] = "Economy"

business = pd.read_csv("business.csv")
business["Type"] = "Business"

df = pd.concat((economy, business))
df.reset_index(inplace=True, drop=True)
df.head()

,date,airline,ch_code,num_code,dep_time,from,time_taken,stop,arr_time,to,price,Type
0,11-02-2022,SpiceJet,SG,8709,18:55,Delhi,02h 10m,non-stop,21:05,Mumbai,"5,953",Economy
1,11-02-2022,SpiceJet,SG,8157,06:20,Delhi,02h 20m,non-stop,08:40,Mumbai,"5,953",Economy
2,11-02-2022,AirAsia,I5,764,04:25,Delhi,02h 10m,non-stop,06:35,Mumbai,"5,956",Economy
3,11-02-2022,Vistara,UK,995,10:20,Delhi,02h 15m,non-stop,12:35,Mumbai,"5,955",Economy
4,11-02-2022,Vistara,UK,963,08:50,Delhi,02h 20m,non-stop,11:10,Mumbai,"5,955",Economy


## Exploration et nettoyage

Commençons par explorer les données. On veut s'assurer de la cohérence des données et identifier des variables qui pourraient être importante dans la modélisation.

**Consigne** : Modifier l'ensemble des noms de colonnes pour qu'ils commençent par une capitale. Puis supprimer les colonnes *Ch_code* et *Num_code*.

Les colonnes *Date*, *Dep_time*, *Time_taken* et *Arr_time* correspondent toute à des dates ou durée. Mettons-les au bon format.

### Travail des dates et des temps

**Consigne** : A l'aide de la fonction [*pd.to_datetime*](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_datetime.html), modifier la colonne *Date* puis créer la colonne *Flight_month* qui contient le mois du vol.

**Consigne** : Remplacer la colonne *Dep_time* par une colonne *Departure_hour* qui ne conserve que l'heure de départ. Faire de même avec la colonne *Arr_time* par *Arrival_hour*.

On souhaite également créer la colonne *Flight_time* à partir de la colonne *Time_taken* qui correspondra à la durée du vol en minutes.

In [5]:
import re

def convert_to_time(string_time):
    pattern = r'(\d+)h (\d+)m'
    match = re.match(pattern, string_time)
    if match:
        hours = int(match.group(1))
        minutes = int(match.group(2))
    else:
        hours, minutes = 0, 0
    return 60 * hours + minutes

df["Flight_time"] = df["Time_taken"].apply(lambda time: convert_to_time(time))
df = df.drop(columns=["Time_taken"], axis=1)

### Autre colonnes

**Consigne** : Explorer la colonne *Stop* et en extraire le nombre d'escale qui sont réalisés. La colonne sera catégorielle.

**Consigne** : A l'aide de la méthode *info*, identifier puis corriger le problème de la colonne *Price*.

**Consigne** : Remplacer la variable *Type* par une valeur 0 ou 1 (pour business). On modifiera le nom de la colonne en conséquence.

**Consigne** : Afficher la représentativité de chaque compagnie aérienne en utilisant la méthode [*value_counts*](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.value_counts.html). Faire de même avec les colonnes *From* et *To* puis commenter.

**Consigne** : A la lumière des observations précédentes, réaliser un One-Hot-Encoding avec la fonction [*pd.get_dummies*](https://pandas.pydata.org/docs/reference/api/pandas.get_dummies.html).

**Consigne** : Trier la totalité du dataset par la colonne *Date* avec la méthode [*sort_values*](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sort_values.html) puis supprimer la colonne *Date*. On prendra soin d'avoir une réindexation du dataset.

## Modélisation

Puisque nous avons un dataset indexé par le temps, nous ne pouvons pas faire de coupure aléatoire.

**Consigne** : Ecrire une fonction *split_time_dataset* qui prend en paramètre
* *X* : une matrice
* *y* : un vecteur
* *train_ratio* : proportion d'observation à placer dans le dataset d'entraînement
La fonction renverra un tuple de même format que la fonction [*train_test_split*](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) de scikit-learn.

In [2]:
def split_time_dataset(X, y, train_ratio):
    ...
    return X_train, X_test, y_train, y_test

**Consigne** : A l'aide la fonction précédente, générer un dataset d'entraînement composé de 75% des observations.

On décide de travailler avec le modèle [LightGBM](https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.LGBMRegressor.html).

**Consigne** : Construire un modèle avec 50 arbres, puis l'entraîner et calculer le vecteur prédit pour le dataset de test.

In [4]:
from lightgbm import LGBMRegressor
...

**Consigne** : Construire une fonction *print_performance* qui prend en paramètre un vecteur *y_true* et un vecteur *y_pred* et qui calcule, puis affiche, la valeur de la RMSE et du R2. Pour aider à l'interprétation, on affichera également la valeur moyenne du vecteur *y_test*.

On sait que les prix sont plus élevés pour les vols business. On veut savoir si le modèle est aussi performant sur les deux types de billet.

**Consigne** : Créer un dataset *result* qui correspond au dataset de test et à l'ajout de la prédiction et du vrai prix.

In [ ]:
result = X_test.copy()
result["y_true"] = ...
result["y_pred"] = ...
result.head()

**Consigne** : Calculer la performance du modèle selon le type de billet. Commenter.

## Effet du learning rate

On veut visualiser l'importance de bien choisir la valeur du learning rate pour le boosting.

**Consigne** : Entraîner plusieurs algorithme avec différents learning rate et comparer les valeurs de RMSE. On produira un graphique, et on considérera un boosting avec 50 arbres.

**Consigne**: Faire la même chose avec un LGBM de 100 arbres et un autre de 250, puis afficher le résultats avec le précédent.

On souhaiterai avoir une vision du sur-apprentissage. Pour ce faire, nous allons mesurer l'écart entre les performances sur le dataset d'entraînement et celui de test.

**Consigne** : Reproduire le graphique précédent en ajoutant l'information de la performance sur le jeu d'entraînement.

## Régularisation : par la profondeur

Il y a un sur-apprentissage légé, et on voudrait pour le réduire. Il existe plusieurs possibilités pour cela, mais nous allons nous concentrer sur la profondeur maximale d'un arbre *max_depth*.

On considère dans la suite un modèle avec 250 arbres, et des learning rates entre $10^{-2}$ et $1$. Puisque nous allons reproduire plusieurs fois des cellules de codes similaires, commençons par en faire une fonction.

**Consigne** : Compléter la fonction *compute_performance* qui prend en paramètres:
* *learning_rates*: une liste de learning rates à tester
* *parameter_name*: le nom du paramètre du modèle que l'on souhaite tester
* *parameter_vales*: une liste de valeur à tester pour le paramètre d'intérêt

In [29]:
def compute_performance(learning_rates, parameter_name, parameter_values, **parameters):
    performances = []
    learning_rates.sort()
    
    for parameter_value in parameter_values:
        performance_train = []
        performance_test = []
        for learning_rate in learning_rates:
            parameters["learning_rate"] = ...
            parameters[...] = ...
            model = LGBMRegressor(**parameters)
            model.fit(..., ...)

            y_pred = model.predict(...)
            performance_train.append(RMSE(..., ...))

            y_pred = model.predict(...)
            performance_test.append(RMSE(..., ...))

        performances.append({parameter_name: parameter_value,
                             "performances_train": performance_train,
                             "performances_test": performance_test})
        
    return performances

Il nous faut maintenant de la visualisation. On utilisera la fonction *plot_performance* ci-dessous.

**Consigne**: À l'aide la fonction *compute_performance* et de la fonction *plot_performance*, visualiser l'impact de la profondeur des arbres en fonction du learning rate.

In [30]:
def plot_performance(performances, learning_rates, parameter_name):
    plt.figure(figsize=(15, 8))
    for index, element in enumerate(performances):
        color = sns.color_palette()[index]
        parameter_value = element[parameter_name]
        powers = -np.log(learning_rates)/np.log(10)
        plt.plot(powers, element["performances_test"], 'o-', label=f"{parameter_name} = {parameter_value}", color=color)
        plt.plot(powers, element["performances_train"], '--', color=color)


    plt.xlabel(r"$-\log(\eta)$")
    plt.ylabel("RMSE")
    plt.ylim(bottom=0)
    plt.title(f"RMSE en fonction du paramètre {parameter_name} pour un LGBM")
    plt.legend()
    plt.show()

## Importance des features

On se pose la question de l'importance de chacune des informations que l'on a donné au modèle. Pour le faire, on doit d'abord entraîner un modèle.

**Consigne** : Avec l'étude préalable, entraîner un modèle avec les  paramètre de votre choix.

**Consigne** : En utilisant la fonction [*plot_importance*](https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.plot_importance.html) de LightGBM, afficher puis commenter l'importance des features.

## Et après ?

On peut s'attaquer à plusieurs axes de réflexion :
1. Comparer les performances entre les principaux algorithmes de boosting
2. Tester d'autres paramètres du boosting
3. Reprendre la préparation des données et l'améliorer.